<a href="https://colab.research.google.com/github/VitalyGladyshev/LLM-engineering/blob/main/LLM_GLVV_HW09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Выбор датасета

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.5 MB/s eta 0

In [ ]:
!pip install pymystem3

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import re

import math
import json
from pathlib import Path

from time import sleep

import docx
from docx import Document
from docx.text.paragraph import Paragraph

from tqdm import tqdm

import time

from pymystem3 import Mystem

random.seed(42)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

import torch
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from chromadb import PersistentClient

from chromadb.utils import embedding_functions
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from sentence_transformers import SentenceTransformer

import chromadb
from uuid import uuid4

In [ ]:
data = pd.DataFrame(columns=['header','text', 'question'])
data

,header,text,question


In [ ]:
def save_to_dataframe(file_path):
    global data

    doc = Document(file_path)
    tab_cnt = 0
    curr_par_len = 0
    chapters = []

    lvl = 0
    number_arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for el in doc.element.body:
        if el.tag.endswith('p'):

            paragraph = Paragraph(el, doc)
            style = paragraph.style.name

            if "Heading" in style:

                hd_parts = style.split()
                curr_lvl = int(hd_parts[-1]) - 1
                number_arr[curr_lvl] += 1

                # Проверим на заполнение номеров предыдущих уровней
                for level in range(curr_lvl):
                    if not number_arr[level]:
                        number_arr[level] = 1

                number = ""
                for i in range(curr_lvl+1):
                    if number:
                        number += "."
                    number += str(number_arr[i])

                while curr_lvl < lvl:
                    number_arr[lvl] = 0
                    lvl -= 1

                lvl = curr_lvl

                print(f"Добавлена глава: {number} {el.text.strip()}")
                # print("  "*curr_lvl + number + " " + el.text.strip())

                curr_par_len = 0

                header = el.text.strip()
                if type(header) == type(0.0):
                    header = ""
                chapters.append([number + " " + header, ""])

            elif paragraph.text:
                curr_par_len += len(paragraph.text)
                if len(chapters):
                    chapters[-1][-1] += paragraph.text
                else:
                    chapters.append([paragraph.text[:20], paragraph.text]) # текст до первого заголовка

        elif el.tag.endswith('tbl'):
            tab_cnt += 1

            table_data = []
            for row in doc.tables[tab_cnt-1].rows:
                row_data = [cell.text.strip() for cell in row.cells]
                table_data.append(row_data)

            if len(chapters):
                chapters[-1][-1] += str(table_data)
            else:
                chapters.append([str(table_data)[:20], str(table_data)])    # таблица до первого заголовка

    for chapter in chapters:
        new_row = {'header': chapter[0], 'text': chapter[1]}
        data = pd.concat([data,
                          pd.DataFrame([new_row], columns=data.columns)],
                         ignore_index = True)
    return data

In [ ]:
data = save_to_dataframe("/content/merged_cr.docx")

Добавлена глава: 1 Назначение программного модуля CAD
Добавлена глава: 2 Условия выполнения программного модуля CAD
Добавлена глава: 3 Выполнение программного модуля CAD
Добавлена глава: 3.1 Запуск программного модуля CAD
Добавлена глава: 3.2 Управление приложениями
Добавлена глава: 3.3 Работа с документами
Добавлена глава: 3.3.1 Создание нового документа
Добавлена глава: 3.3.2 Открытие документов
Добавлена глава: 3.3.3 Сохранение документов
Добавлена глава: 3.3.4 Закрытие документа
Добавлена глава: 3.3.5 Настройка документов
Добавлена глава: 3.3.5.1 Свойства документа
Добавлена глава: 3.3.5.2 Атрибуты документа
Добавлена глава: 3.3.5.3 История документа
Добавлена глава: 3.3.5.4 Стили объектов
Добавлена глава: 3.4 Справочная система
Добавлена глава: 4 Интерфейс
Добавлена глава: 4.1 Окна программного модуля CAD
Добавлена глава: 4.1.1 Рабочие окна
Добавлена глава: 4.1.2 Вспомогательные окна
Добавлена глава: 4.2 Инструментальные панели
Добавлена глава: 4.2.1 Панель «Лента»
Добавлена глава

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   header    44 non-null     object
 1   text      44 non-null     object
 2   question  0 non-null      object
dtypes: object(3)
memory usage: 1.2+ KB


In [ ]:
data.head()

,header,text,question
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,NaN
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,NaN
2,3 Выполнение программного модуля CAD,,NaN
3,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,NaN
4,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,NaN


In [ ]:
data = data.loc[data.text != ""]
data.reset_index(drop=True, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   header    40 non-null     object
 1   text      40 non-null     object
 2   question  0 non-null      object
dtypes: object(3)
memory usage: 1.1+ KB


In [ ]:
data.head()

,header,text,question
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,NaN
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,NaN
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,NaN
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,NaN
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,NaN


# 2. Подготовка датасета

In [ ]:
from openai import OpenAI

from google.colab import userdata

MODEL_CONFIG = {
    "base_url": "https://api.vsegpt.ru/v1",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "model_name": "openai/gpt-5", #-mini", # "openai/gpt-5-nano"
    "temperature": 0.5,
    "top_p": 0.9
}

In [ ]:
class LLM:
    def __init__(self):
        self.llm = OpenAI(base_url=MODEL_CONFIG["base_url"], api_key=MODEL_CONFIG["api_key"])
        self.system_prompt = "Ты выполняеешь задание пользователя. Не добавляй лишних слов, выводи только результат"

    def set_system_prompt(self, system_prompt: str):
        """Установка системного промпта"""
        self.system_prompt = system_prompt

    def generate_answer(self, question: str) -> str:
        """Генерация ответа модели"""

        # no_think = "/no_think\n"
        messages = []
        messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": question}) # no_think +

        reply = self.llm.chat.completions.create(
            model=MODEL_CONFIG["model_name"],
            messages=messages,
            temperature=MODEL_CONFIG["temperature"],
            top_p=MODEL_CONFIG["top_p"],
            stream=False
        )

        return reply.choices[0].message.content #[19:]

In [ ]:
llm = LLM()

In [ ]:
llm.generate_answer("Напиши последовательно числа от 1 до 10")

'1 2 3 4 5 6 7 8 9 10'

In [ ]:
# llm.set_system_prompt(
#             "Ты участвуешь в решении задачи оценки конвейра RAG. "
#             "Ты создаёшь набор данных для оценки семантического поиска. "
#             "Задай простой, но содержательный вопрос к тексту, которые мог бы задать пользователь"
#         )
llm.set_system_prompt(
            "Ты участвуешь в решении задачи оценки конвейра RAG. "
            "Ты создаёшь набор данных для оценки семантического поиска. "
            "Задай вопрос к тексту с использованием лексики близкой, но неидентичной лексике текста, "
            "так чтобы протестировать систему поиска - чтобы сравнивать различные алгоритмы поиска"
        )

In [ ]:
for i in tqdm(range(data.shape[0])):
    if data.iloc[i]["text"]:
        prompt = (
            f"Представь себя пользователем RAG системы. "
            f"Создай вопрос к тексту с использованием лексики близкой, но неидентичной лексике текста."
            f"Вопрос должен быть сложным для системы поиска, но выполнимым - чтобы сравнивать различные алгоритмы поиска."
            f"\nТекст:\n\n{data.iloc[i]['text']}"
        )

        data.loc[i, 'question'] = llm.generate_answer(prompt)

100%|██████████| 40/40 [14:40<00:00, 22.01s/it]


In [ ]:
data.head()

,header,text,question
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,На какой интегрированной инженерной основе реа...
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,Какие минимальные пороги по окружению и “желез...
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,Какими действиями из интерфейсной оболочки RPL...
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,Какими двумя способами вызвать диспетчер допол...
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,В каком разделе главного меню собран консолиди...


In [ ]:
data.tail()

,header,text,question
35,5.1.7 Сведения об объекте,Сведения о выбранном объекте или наборе объект...,Какие реквизиты отображаются в панели сведений...
36,5.1.8 Управление объектами через слои,Слой – параметр каждого геометрического объект...,"Как в окне диспетчера уровней, открываемом с л..."
37,5.1.9 Управление приоритетом объектов,По умолчанию очерёдность отображения перекрыва...,Если нужно разместить выбранный 2D‑элемент сра...
38,5.1.10 Сцены,Команда «Создать сцену» предназначена для созд...,Как именно инициируется создание новой 3D‑сцен...
39,5.1.11 Управление представлениями,Управление представлениями рабочего окна предп...,Во время «орбиты» сцены с зажатой ПКМ: какая т...


In [ ]:
data.text[0]

'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными параметрами отображения.'

In [ ]:
data.question[0]

'На какой интегрированной инженерной основе реализован этот CAD‑компонент (с указанием применяемого 3D‑вычислительного ядра), и какие именно операции он поддерживает с геометрией и сопутствующими метаданными — от создания/корректировки 2D/3D‑представлений и назначения PMI‑аннотаций (допуски, размеры, технологические характеристики) до выпуска конструкторских материалов по нормам ЕСКД и интерактивного просмотра с настраиваемыми режимами визуализации?'

In [ ]:
data.to_csv("parsed_q.csv", index=False)

In [ ]:
SIZE = 350

In [ ]:
data_chunks = pd.DataFrame(columns=['id_ch','text_ch', 'text_ch_prep', 'chapter_index', 'part_index'])
data_parts = pd.DataFrame(columns=['id_part','text_part'])

In [ ]:
documents = []
metadatas = []
ids = []

part_cnt = 0

for i in range(data.shape[0]):

    text_len = len(data.iloc[i]["text"])

    if data.iloc[i]["text"] == "-" or data.iloc[i]["text"] != data.iloc[i]["text"]:
        continue

    elif text_len <= SIZE:
        print(f"{i} длина: {len(data.iloc[i]['text'])}")

        part_id = str(uuid4())
        chunk_id = str(uuid4())

        documents.append(f"{data.iloc[i]['header'].strip()}\n{data.iloc[i]['text'].strip()}")
        metadatas.append({"header": data['header'][i],
                          "index": str(i),
                          "chunk":  "",
                          "content": data.iloc[i]["text"].strip(),
                          "id_ch": chunk_id,
                          "chapter_index": str(i)
                         })
        ids.append(chunk_id)
        new_row = {'id_part': part_id, 'text_part': data.iloc[i]["text"].strip()}
        data_parts = pd.concat([data_parts,
                                pd.DataFrame([new_row], columns=data_parts.columns)],
                               ignore_index = True)

        new_row = {'id_ch': str(uuid4()),
                   'text_ch': data.iloc[i]["text"].strip(),
                   'text_ch_prep': "",
                   'part_index': part_id,
                   'chapter_index': str(i)}
        data_chunks = pd.concat([data_chunks,
                                pd.DataFrame([new_row], columns=data_chunks.columns)],
                               ignore_index = True)
        part_cnt += 1

    else:
        print(f"{i}")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=SIZE,
            chunk_overlap=0,
            separators=["\n", " ", ""]  # "\n\n", Порядок приоритета разделителей
        )

        chunks = text_splitter.split_text(data.iloc[i]["text"].strip())

        len_ch = len(chunks)
        step = 13
        if len_ch > step:
            for j in range(step, len_ch, step):
                part = " ".join(chunks[j-(step):j])
                part_id = str(uuid4())
                for k in range(j - step, j):
                    chunk_id = str(uuid4())

                    documents.append(f"{data['header'][i].strip()}\n{chunks[k].strip()}")
                    metadatas.append({"header": data['header'][i],
                                      "index": str(i),
                                      "chunk":  chunks[k],
                                      "content": part.strip(),
                                      "id_ch": chunk_id,
                                      "chapter_index": str(i)
                                     })
                    ids.append(chunk_id)

                    new_row = {'id_ch': str(uuid4()),
                               'text_ch': chunks[k],
                               'text_ch_prep': "",
                               'part_index': part_id,
                               'chapter_index': str(i)}
                    data_chunks = pd.concat([data_chunks,
                                            pd.DataFrame([new_row], columns=data_chunks.columns)],
                                            ignore_index = True)
                    print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")

                new_row = {'id_part': part_id, 'text_part': part.strip()}
                data_parts = pd.concat([data_parts,
                                        pd.DataFrame([new_row], columns=data_parts.columns)],
                                    ignore_index = True)
                part_cnt += 1
        else:
            j = 0

        if j < len_ch or len_ch < step:
            part = " ".join(chunks[j:])
            part_id = str(uuid4())
            for k in range(j, len_ch):
                chunk_id = str(uuid4())

                documents.append(f"{data['header'][i].strip()}\n{chunks[k].strip()}")
                metadatas.append({"header": data['header'][i],
                                  "index": str(i),
                                  "chunk":  chunks[k],
                                  "content": part.strip(),
                                  "id_ch": chunk_id,
                                  "chapter_index": str(i)
                                 })
                ids.append(chunk_id)

                new_row = {'id_ch': str(uuid4()),
                           'text_ch': chunks[k],
                           'text_ch_prep': "",
                           'part_index': part_id,
                           'chapter_index': str(i)}
                data_chunks = pd.concat([data_chunks,
                                        pd.DataFrame([new_row], columns=data_chunks.columns)],
                                        ignore_index = True)
                print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")

            new_row = {'id_part': part_id, 'text_part': part.strip()}
            data_parts = pd.concat([data_parts,
                                    pd.DataFrame([new_row], columns=data_parts.columns)],
                                   ignore_index = True)
            part_cnt += 1

0
part len: 598 chunk len: 343
part len: 598 chunk len: 254
1 длина: 347
2
part len: 825 chunk len: 328
part len: 825 chunk len: 349
part len: 825 chunk len: 146
3
part len: 1297 chunk len: 342
part len: 1297 chunk len: 343
part len: 1297 chunk len: 344
part len: 1297 chunk len: 265
4 длина: 73
5
part len: 1068 chunk len: 344
part len: 1068 chunk len: 345
part len: 1068 chunk len: 349
part len: 1068 chunk len: 27
6
part len: 2004 chunk len: 349
part len: 2004 chunk len: 343
part len: 2004 chunk len: 339
part len: 2004 chunk len: 347
part len: 2004 chunk len: 342
part len: 2004 chunk len: 279
7
part len: 1177 chunk len: 344
part len: 1177 chunk len: 346
part len: 1177 chunk len: 348
part len: 1177 chunk len: 136
8 длина: 65
9
part len: 1272 chunk len: 342
part len: 1272 chunk len: 335
part len: 1272 chunk len: 334
part len: 1272 chunk len: 258
10
part len: 2247 chunk len: 350
part len: 2247 chunk len: 335
part len: 2247 chunk len: 341
part len: 2247 chunk len: 341
part len: 2247 chunk l

In [ ]:
data_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_ch          221 non-null    object
 1   text_ch        221 non-null    object
 2   text_ch_prep   221 non-null    object
 3   chapter_index  221 non-null    object
 4   part_index     221 non-null    object
dtypes: object(5)
memory usage: 8.8+ KB


In [ ]:
data_chunks.tail()

,id_ch,text_ch,text_ch_prep,chapter_index,part_index
216,2c4ea2f2-087f-4349-8936-0f194a3f2670,"пересечение луча, проходящего через точку захв...",,39,07da45fb-bf82-42f5-83ec-ba32f23324a1
217,bd469e64-858c-4d8c-ae81-1e7e069297d6,графического окна при захвате СКМ;масштабирова...,,39,07da45fb-bf82-42f5-83ec-ba32f23324a1
218,c8137c9d-f35c-424f-98a7-20b1977c7c2f,CAD.Для управления 3Dсценой в графическом окне...,,39,07da45fb-bf82-42f5-83ec-ba32f23324a1
219,45ef6268-e589-4dfe-b919-47f91df95a10,определяет положение точки обзора сцены. Пикто...,,39,07da45fb-bf82-42f5-83ec-ba32f23324a1
220,2748a9a7-41fd-4c1d-8c24-38d907e3c88b,3Dсценой может осуществляться с помощью 3Dмани...,,39,07da45fb-bf82-42f5-83ec-ba32f23324a1


In [ ]:
data_parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_part    43 non-null     object
 1   text_part  43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [ ]:
data_parts.tail()

,id_part,text_part
38,954e2b9d-e658-42fa-8605-857a6db17ed2,Сведения о выбранном объекте или наборе объект...
39,aaa33ac0-e31a-4fce-abf7-291e8fe9528f,Слой – параметр каждого геометрического объект...
40,c83ead59-63b8-4b59-bb9f-56889e8f8902,По умолчанию очерёдность отображения перекрыва...
41,f63f5472-753b-4b02-adf2-ea05f745ccd6,Команда «Создать сцену» предназначена для созд...
42,07da45fb-bf82-42f5-83ec-ba32f23324a1,Управление представлениями рабочего окна предп...


In [ ]:
data_parts.to_csv("parts.csv", index=False)

In [ ]:
mystem_analyzer = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

russian_stopwords = stopwords.words('russian')
print(russian_stopwords[:10])

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def chunk_preparation(text: str) -> str:
    """Препроцессинг текста для BM25"""
    if pd.isna(text) or text == "":
        return ""

    prep_text = text.lower()
    prep_text = prep_text.replace("\n", " ")
    prep_text = prep_text.replace("..", ".")
    prep_text = re.sub(r"@\w*", "", prep_text)
    prep_text = re.sub("[^а-яА-ЯёЁ0-9 ]", " ", prep_text)
    prep_text = mystem_analyzer.lemmatize(prep_text)

    # Фильтрация стоп-слов и пустых токенов
    prep_text = [word for word in prep_text if word not in russian_stopwords and word.strip()]

    return ' '.join(prep_text)

In [ ]:
data_chunks['text_ch_prep'] = data_chunks['text_ch'].apply(chunk_preparation)

In [ ]:
data_chunks.head()

,id_ch,text_ch,text_ch_prep,chapter_index,part_index
0,98b16826-8a86-4e13-8332-c26f886cc402,Программный модуль CAD разработан на базе инте...,программный модуль разрабатывать база интегрир...,0,475e075f-0cc7-4f5f-aeab-54f98f9634df
1,b93da151-8c7f-45e0-b8b2-aa2708df007d,"связанной с точностными, размерными, технологи...",связывать точностный размерный технологический...,0,475e075f-0cc7-4f5f-aeab-54f98f9634df
2,d3a6a27c-3064-4a2b-bbbe-4c31e549e6ff,Достаточными условиями выполнения программного...,достаточный условие выполнение программный мод...,1,2fca5495-486b-4dd7-ab4f-95a77c5cc00f
3,a3c4412d-17bd-4520-8c2f-ff182a08aacf,Загрузка и запуск программного модуля CAD осущ...,загрузка запуск программный модуль осуществлят...,2,7bcfba84-f904-4bba-9be2-681abe80439a
4,686e0814-48f6-4b77-916b-05649760aa2b,«Приветствие»Рисунок Данная страница содержит ...,приветствие рисунок данный страница содержать ...,2,7bcfba84-f904-4bba-9be2-681abe80439a


In [ ]:
data_chunks.to_csv("chunks.csv", index=False)

# 3. Подготовка функции для оценки качества поиска

In [ ]:
class SemanticSearchEvaluation:

    def __init__(
        self,
        emb #_path: str

    ):
        # Загружаем эмбеддинги
        # self.emb_path = emb_path

        # Проверка доступности GPU и установка устройства
        # self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # print(f"Using device: {self.device}")

        self.embedding_function = sentence_transformer_ef_cl
        # self.embedding_function = HuggingFaceEmbeddings(
        #     model_name=self.emb_path,
        #     model_kwargs={'device': self.device},
        #     encode_kwargs={
        #         'normalize_embeddings': True,
        #         'convert_to_tensor': True
        #     },
        #     show_progress=False
        # )
        self.result = dict()
        self.collection_name = "data_collection"

    def _get_best_device(self) -> str:
        """Определяет оптимальное устройство для вычислений (GPU/CPU)."""
        if torch.cuda.is_available():
            print(f"GPU доступен: {torch.cuda.get_device_name(0)}")
            return "cuda"
        else:
            print("GPU недоступен, используется CPU")
            return "cpu"

    def evaluate(self,
                 db_path: str,
                 data: pd.DataFrame,
                 column_name: str,
                 displacement: int=0):
        # Загружаем базу Chroma
        # Создаём клиента
        parent_folder = Path(os.getcwd()) # Path(__file__).parent.parent
        path = str(parent_folder / db_path)
        # print(f"path: {path}")

        self.client = PersistentClient(path=path)

        # Инициализация векторного хранилища с embedding функцией
        self.vector_store = Chroma(
            client=self.client,
            collection_name=self.collection_name,
            embedding_function=self.embedding_function,
            collection_metadata={"hnsw:space": "cosine"}
        )

        # Получаем коллекцию для прямых операций
        self.collection = self.vector_store._collection

        # датасет
        df = data

        total = 0    # Вопросов всего
        correct = 0  # Правильных индексов (первый верный)
        both = 0   # Оба индекса верные
        present = 0  # Индекс есть в списке из 5

        for i in tqdm(range(displacement, df.shape[0])):
            if df.iloc[i][column_name] == "-" or df.iloc[i][column_name] != df.iloc[i][column_name]:
                continue

            curr_indexes = []
            vect_res = self.vector_store.similarity_search(
                df.iloc[i][column_name],
                k=5
            )
            len_vec = len(vect_res)
            for j in range(len_vec):
                curr_indexes.append(int(vect_res[j].metadata['index']))
            # print(f"i: {i} curr_indexes: {curr_indexes}")
            total += 1
            if len(curr_indexes) and i == curr_indexes[0]:
                correct += 1
            if len(curr_indexes) and (i == curr_indexes[0] or i == curr_indexes[1]):
                both += 1
            if i in curr_indexes:
                present += 1

        self.result["total"] = total
        self.result["correct"] = correct
        self.result["both"] = both
        self.result["present"] = present
        self.result["top1"] = self.result["correct"]/self.result["total"]*100
        self.result["top2"] = self.result["both"]/self.result["total"]*100
        self.result["top5"] = self.result["present"]/self.result["total"]*100

        return self.result

    def show_result(self):
        print(f'Вопросов всего: {self.result["total"]}')
        print(f'Правильных индексов (первый верный): {self.result["correct"]} доля: {self.result["top1"]:.2f}%')
        print(f'Первый или второй верные: {self.result["both"]} доля: {self.result["top2"]:.2f}%')
        print(f'Индекс есть в списке из 5: {self.result["present"]} доля: {self.result["top5"]:.2f}%')

# 4. Dense поиск

In [ ]:
data = pd.read_csv("parsed_q.csv")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   header    40 non-null     object
 1   text      40 non-null     object
 2   question  40 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [ ]:
data_chunks = pd.read_csv("chunks.csv")

In [ ]:
data_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_ch          221 non-null    object
 1   text_ch        221 non-null    object
 2   text_ch_prep   221 non-null    object
 3   chapter_index  221 non-null    int64 
 4   part_index     221 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.8+ KB


In [ ]:
data_chunks.head()

,id_ch,text_ch,text_ch_prep,chapter_index,part_index
0,98b16826-8a86-4e13-8332-c26f886cc402,Программный модуль CAD разработан на базе инте...,программный модуль разрабатывать база интегрир...,0,475e075f-0cc7-4f5f-aeab-54f98f9634df
1,b93da151-8c7f-45e0-b8b2-aa2708df007d,"связанной с точностными, размерными, технологи...",связывать точностный размерный технологический...,0,475e075f-0cc7-4f5f-aeab-54f98f9634df
2,d3a6a27c-3064-4a2b-bbbe-4c31e549e6ff,Достаточными условиями выполнения программного...,достаточный условие выполнение программный мод...,1,2fca5495-486b-4dd7-ab4f-95a77c5cc00f
3,a3c4412d-17bd-4520-8c2f-ff182a08aacf,Загрузка и запуск программного модуля CAD осущ...,загрузка запуск программный модуль осуществлят...,2,7bcfba84-f904-4bba-9be2-681abe80439a
4,686e0814-48f6-4b77-916b-05649760aa2b,«Приветствие»Рисунок Данная страница содержит ...,приветствие рисунок данный страница содержать ...,2,7bcfba84-f904-4bba-9be2-681abe80439a


In [ ]:
data_parts = pd.read_csv("parts.csv")

In [ ]:
data_parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_part    43 non-null     object
 1   text_part  43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device for embedding functions: {device}")

Using device for embedding functions: cpu


In [ ]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-large", device=device)
sentence_transformer_ef_cl = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large", model_kwargs={'device': device})

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
persistent_client_serv = chromadb.PersistentClient(path="./db_data")
collection_serv = persistent_client_serv.get_or_create_collection("data_collection", embedding_function=sentence_transformer_ef)

In [ ]:
# ids = data_chunks.id_ch.values.tolist()

In [ ]:
len(documents), len(metadatas), len(ids)

(221, 221, 221)

In [ ]:
start = time.time()

collection_serv.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas
)

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

Время исполнения: 260.061 с.


In [ ]:
start_time = time.time()

vector_store_serv_from_client = Chroma(
    client=persistent_client_serv,
    collection_name="data_collection",
    embedding_function=sentence_transformer_ef_cl,
)

response_time = time.time() - start_time
print(f"Время построения клиента: {response_time:.3f} секунды")

Время построения клиента: 0.004 секунды


In [ ]:
question = "Программный модуль"

In [ ]:
vect_res = vector_store_serv_from_client.similarity_search(
    question,
    k=5
)

In [ ]:
for i in range(len(vect_res)):
    print(f"{vect_res[i].page_content}\n")

1 Назначение программного модуля CAD
связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными параметрами отображения.

4 Интерфейс
временный файл формата XML, из которого происходит восстановление настроек при повторном запуске программы.Программный модуль CAD обеспечивает возможность сохранения настроек интерфейса в настроечный файл формата INI в виде профиля, который может использоваться для передачи на другие рабочие места или в другую конфигурацию программного модуля

3.2 Управление приложениями
диалог операционной системы «Открыть».Отключение модуля осуществляется после его выбора в перечне подключенных с помощью кнопки «Удалить».Изменение состояния модулей происходит после перезагрузки системы.Для автоматического запуска программного компонента вместе с системой, необходимо установить флаг в графе «Автозапуск

In [ ]:
vect_res[0]

Document(id='28204f81-a391-4a19-8d61-e3c626d255e8', metadata={'chapter_index': '0', 'chunk': 'связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными параметрами отображения.', 'header': '1 Назначение программного модуля CAD', 'index': '0', 'content': 'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными

In [ ]:
docs = []
for i in range(len(vect_res)):
    if vect_res[i].metadata["content"] not in docs:
        docs.append(vect_res[i].metadata["content"])
        print(f"{i+1} {vect_res[i].metadata["content"]}\n")

1 Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными параметрами отображения.

2 Пользовательский интерфейс представляется собой главное окно программного модуля CAD, которое включает в себя следующие основные элементы (рис. Рисунок):панель быстрого доступа с набором часто используемых команд и полем поиска команд (1);выпадающее меню «Файл» с набором инструментов для работы с документами (2);структурно-иерархическая панель инструментов «Лента» с контекстно-зависимыми набор

In [ ]:
!nvidia-smi

Sun Oct 19 14:44:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             30W /   70W |    4692MiB /  15360MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
retriver_test = SemanticSearchEvaluation(sentence_transformer_ef_cl) #"intfloat/multilingual-e5-large")

res = retriver_test.evaluate("db_data", data, "question")
res

100%|██████████| 40/40 [00:59<00:00,  1.49s/it]


{'total': 40,
 'correct': 29,
 'both': 34,
 'present': 39,
 'top1': 72.5,
 'top2': 85.0,
 'top5': 97.5}

In [ ]:
retriver_test.show_result()

Вопросов всего: 40
Правильных индексов (первый верный): 29 доля: 72.50%
Первый или второй верные: 34 доля: 85.00%
Индекс есть в списке из 5: 39 доля: 97.50%


# 5. Sparse поиск

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
tokenized_corpus = [doc.split(" ") for doc in data_chunks['text_ch_prep'].tolist()]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
data.question[0]

'На какой интегрированной инженерной основе реализован этот CAD‑компонент (с указанием применяемого 3D‑вычислительного ядра), и какие именно операции он поддерживает с геометрией и сопутствующими метаданными — от создания/корректировки 2D/3D‑представлений и назначения PMI‑аннотаций (допуски, размеры, технологические характеристики) до выпуска конструкторских материалов по нормам ЕСКД и интерактивного просмотра с настраиваемыми режимами визуализации?'

In [ ]:
chunk_preparation(data.question[0])

'интегрировать инженерный основа реализовывать компонент указание применять 3 вычислительный ядро именно операция поддерживать геометрия сопутствующий метаданные создание корректировка 2 3 представление назначение аннотация допуск размер технологический характеристика выпуск конструкторский материал норма ескд интерактивный просмотр настраивать режим визуализация'

In [ ]:
def bm25_doc_scores(query, top_k=10):
    scores = bm25.get_scores(chunk_preparation(query))
    # print(f"scores: {scores}")
    top_idx = np.argsort(scores)[::-1][:top_k]
    # print(f"top_idx: {top_idx}")
    return {int(i): float(scores[i]) for i in top_idx}, {float(scores[i]): int(i) for i in top_idx}

In [ ]:
data.question[5]

'После запуска команды создания нового файла с 3D-содержимым какая панель/мастер выбора шаблона открывается, чем именно отличаются предлагаемые заготовки (укажите, какие стартовые элементы в них присутствуют, какие функциональные настройки модели заданы и какое рабочее окно — плоское или объемное — откроется первым), и какие действия нужно выполнить, чтобы добавить собственную заготовку в этот список: какой базовый бланк следует открыть, какие ориентации осей и стандартные проекции настроить, какие стили и реквизиты новых объектов задать, в какой каталог сохранить результат (приведите полный путь), и в какой конфигурационный ini-файл нужно внести сведения о новой заготовке, чтобы она появилась в окне выбора?'

In [ ]:
data_chunks.text_ch[112]

'многострочными, т.е. предполагают выбор одного либо группы объектов соответственно. В незаполненных полях отображаются подсказки о типе выбираемых объектов. Перед выбором объектов необходимо активировать целевое поле. Активное поле в этом случае будет подсвечено\xa0(рис.\xa0Рисунок).Диалог параметров командРисунок Отмена выбора объекта в однострочном'

In [ ]:
i2s, s2i = bm25_doc_scores(data.question[5])
i2s, s2i

({112: 205.02332805628893,
  66: 197.84945923101318,
  91: 197.84945923101318,
  163: 166.58447385246052,
  164: 165.25816466330815,
  25: 7.399353139431204,
  220: 3.590938549176446,
  134: 2.9989557592742857,
  9: 2.8478162808062137,
  59: 2.6773368610030928},
 {205.02332805628893: 112,
  197.84945923101318: 91,
  166.58447385246052: 163,
  165.25816466330815: 164,
  7.399353139431204: 25,
  3.590938549176446: 220,
  2.9989557592742857: 134,
  2.8478162808062137: 9,
  2.6773368610030928: 59})

In [ ]:
data_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_ch          442 non-null    object
 1   text_ch        442 non-null    object
 2   text_ch_prep   442 non-null    object
 3   chapter_index  442 non-null    object
 4   part_index     442 non-null    object
dtypes: object(5)
memory usage: 17.4+ KB


In [ ]:
max(s2i.keys()), s2i[max(s2i.keys())], int(data_chunks.chapter_index[s2i[max(s2i.keys())]])

(205.02332805628893, 112, 27)

In [ ]:
for k, v in s2i.items():
    print(f"{k}: {v} chapter: {int(data_chunks.chapter_index[v])}")

205.02332805628893: 112 chapter: 27
197.84945923101318: 91 chapter: 25
166.58447385246052: 163 chapter: 33
165.25816466330815: 164 chapter: 33
7.399353139431204: 25 chapter: 8
3.590938549176446: 220 chapter: 39
2.9989557592742857: 134 chapter: 28
2.8478162808062137: 9 chapter: 3
2.6773368610030928: 59 chapter: 16


In [ ]:
class BM25SearchEvaluation:

    def __init__(
        self,
        bm25

    ):
        self.bm25 = bm25
        self.result = dict()

    def evaluate(self,
                 data: pd.DataFrame,
                 column_name: str,
                 displacement: int=0):

        # датасет
        df = data

        total = 0    # Вопросов всего
        correct = 0  # Правильных индексов (первый верный)
        both = 0   # Оба индекса верные
        present = 0  # Индекс есть в списке из 5

        for i in tqdm(range(displacement, df.shape[0])):
            if df.iloc[i][column_name] == "-" or df.iloc[i][column_name] != df.iloc[i][column_name]:
                continue

            curr_indexes = []

            # print(df.iloc[i][column_name])
            i2s, s2i = bm25_doc_scores(df.iloc[i][column_name], top_k=5)
            # print(f"i2s: {i2s}")

            for k, v in s2i.items():
                curr_indexes.append(int(data_chunks.chapter_index[v]))

            # print(f"i: {i} curr_indexes: {curr_indexes}")
            total += 1
            if len(curr_indexes) and i == curr_indexes[0]:
                correct += 1
            if len(curr_indexes) and (i == curr_indexes[0] or i == curr_indexes[1]):
                both += 1
            if i in curr_indexes:
                present += 1

        self.result["total"] = total
        self.result["correct"] = correct
        self.result["both"] = both
        self.result["present"] = present
        self.result["top1"] = self.result["correct"]/self.result["total"]*100
        self.result["top2"] = self.result["both"]/self.result["total"]*100
        self.result["top5"] = self.result["present"]/self.result["total"]*100

        return self.result

    def show_result(self):
        print(f'Вопросов всего: {self.result["total"]}')
        print(f'Правильных индексов (первый верный): {self.result["correct"]} доля: {self.result["top1"]:.2f}%')
        print(f'Первый или второй верные: {self.result["both"]} доля: {self.result["top2"]:.2f}%')
        print(f'Индекс есть в списке из 5: {self.result["present"]} доля: {self.result["top5"]:.2f}%')

In [ ]:
bm25_evaluation = BM25SearchEvaluation(bm25)

res = bm25_evaluation.evaluate(data, "question")
res

100%|██████████| 40/40 [00:01<00:00, 33.65it/s]


{'total': 40,
 'correct': 1,
 'both': 2,
 'present': 3,
 'top1': 2.5,
 'top2': 5.0,
 'top5': 7.5}

In [ ]:
bm25_evaluation.show_result()

Вопросов всего: 40
Правильных индексов (первый верный): 1 доля: 2.50%
Первый или второй верные: 2 доля: 5.00%
Индекс есть в списке из 5: 3 доля: 7.50%


In [ ]:
MODEL_CONFIG = {
    "base_url": "https://api.vsegpt.ru/v1",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "model_name": "openai/gpt-5-mini", # "openai/gpt-5-nano"
    "temperature": 0.3,
    "top_p": 0.9
}

In [ ]:
llm = LLM()

In [ ]:
llm.set_system_prompt(
            "Ты участвуешь в решении задачи оценки конвейра RAG. "
            "Ты создаёшь набор данных для оценки семантического поиска. "
            "Задай вопрос с использованием лексики текста."
        )

In [ ]:
for i in tqdm(range(data.shape[0])):
    if data.iloc[i]["text"]:
        prompt = (
            f"Представь себя пользователем RAG системы. "
            f"Задай вопрос с использованием лексики текста."
            f"\nТекст:\n\n{data.iloc[i]['text']}"
        )

        data.loc[i, 'question'] = llm.generate_answer(prompt)

100%|██████████| 40/40 [05:51<00:00,  8.79s/it]


In [ ]:
retriver_test = SemanticSearchEvaluation(sentence_transformer_ef_cl)

res = retriver_test.evaluate("db_data", data, "question")
res

100%|██████████| 40/40 [00:01<00:00, 33.22it/s]


{'total': 40,
 'correct': 39,
 'both': 40,
 'present': 40,
 'top1': 97.5,
 'top2': 100.0,
 'top5': 100.0}

In [ ]:
retriver_test.show_result()

Вопросов всего: 40
Правильных индексов (первый верный): 39 доля: 97.50%
Первый или второй верные: 40 доля: 100.00%
Индекс есть в списке из 5: 40 доля: 100.00%


In [ ]:
bm25_evaluation = BM25SearchEvaluation(bm25)

res = bm25_evaluation.evaluate(data, "question")
res

100%|██████████| 40/40 [00:00<00:00, 84.17it/s]


{'total': 40,
 'correct': 1,
 'both': 2,
 'present': 3,
 'top1': 2.5,
 'top2': 5.0,
 'top5': 7.5}

In [ ]:
bm25_evaluation.show_result()

Вопросов всего: 40
Правильных индексов (первый верный): 1 доля: 2.50%
Первый или второй верные: 2 доля: 5.00%
Индекс есть в списке из 5: 3 доля: 7.50%


# 6. Hybrid поиск

In [ ]:
def hybrid_scores(query,
                  dense_retriever,
                  sparse_retriever,
                  data_chunks,
                  alpha=0.5,
                  top_k_dense=100,
                  top_k_sparse=100,
                  top_k_fused=100):
    """
    Гибридный поиск с комбинацией плотного и разреженного поиска

    Args:
        query: поисковый запрос
        dense_retriever: ChromaDB retriever
        sparse_retriever: BM25Okapi объект
        data_chunks: DataFrame с чанками
        alpha: вес для плотного поиска (0-1)
        top_k_dense: количество результатов из плотного поиска
        top_k_sparse: количество результатов из разреженного поиска
        top_k_fused: количество финальных результатов

    Returns:
        dict: {id_ch: fused_score} отсортированный по убыванию оценки
    """

    # 1. ПЛОТНЫЙ ПОИСК (Dense/Semantic Search)
    dense_results = dense_retriever.similarity_search_with_score(query, k=top_k_dense)
    dscores = {}

    for doc, score in dense_results:
        doc_id = doc.metadata.get('id_ch')
        if doc_id is not None:
            # Для косинусного расстояния или L2: меньше = лучше
            # Преобразуем в similarity: используем обратное значение
            similarity_score = 1 / (1 + score)  # или -score для простоты
            dscores[doc_id] = similarity_score

    # 2. РАЗРЕЖЕННЫЙ ПОИСК (BM25)
    # Препроцессинг запроса
    query_prep = chunk_preparation(query)
    query_tokens = query_prep.split()

    # Получение BM25 оценок
    bm25_scores_raw = sparse_retriever.get_scores(query_tokens)
    top_indices_sparse = np.argsort(bm25_scores_raw)[::-1][:top_k_sparse]

    bscores = {}
    for idx in top_indices_sparse:
        if idx < len(data_chunks):
            doc_id_sparse = data_chunks.iloc[idx]['id_ch']
            raw_bm25_score = bm25_scores_raw[idx]
            bscores[doc_id_sparse] = raw_bm25_score

    # 3. ОБЪЕДИНЕНИЕ МНОЖЕСТВА ДОКУМЕНТОВ
    all_docs = set(dscores.keys()) | set(bscores.keys())

    if not all_docs:
        return {}

    # 4. Z-НОРМАЛИЗАЦИЯ
    def z_norm(scores_dict):
        if not scores_dict or len(scores_dict) == 0:
            return {}
        scores_array = np.array(list(scores_dict.values()))
        mu = scores_array.mean()
        std = scores_array.std()

        if std == 0:  # Все оценки одинаковые
            return {k: 0.0 for k in scores_dict.keys()}

        z_normalized = (scores_array - mu) / (std + 1e-10)
        return dict(zip(scores_dict.keys(), z_normalized))

    dz = z_norm(dscores)
    bz = z_norm(bscores)

    # 5. СЛИЯНИЕ ОЦЕНОК
    fused_scores = {}
    for doc_id in all_docs:
        dense_norm_score = dz.get(doc_id, 0.0)
        sparse_norm_score = bz.get(doc_id, 0.0)
        fused_score = alpha * dense_norm_score + (1 - alpha) * sparse_norm_score
        fused_scores[doc_id] = fused_score

    # 6. СОРТИРОВКА И ВОЗВРАТ TOP-K
    sorted_fused = dict(
        sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)[:top_k_fused]
    )

    return sorted_fused

In [ ]:
class HybridSearchEvaluation:
    """Класс для оценки гибридного поиска"""

    def __init__(self, dense_retriever, sparse_retriever, data_chunks):
        self.dense_retriever = dense_retriever
        self.sparse_retriever = sparse_retriever
        self.data_chunks = data_chunks
        self.result = {}

        # Создаем маппинг id_ch -> chapter_index для быстрого поиска
        self.id_to_chapter_map = dict(
            zip(self.data_chunks['id_ch'], self.data_chunks['chapter_index'])
        )

    def evaluate(self,
                 data: pd.DataFrame,
                 column_name: str,
                 alpha=0.5,
                 top_k=5,
                 displacement: int = 0):
        """
        Оценка качества гибридного поиска

        Args:
            data: DataFrame с вопросами и ожидаемыми индексами глав
            column_name: название столбца с вопросами
            alpha: вес плотного поиска (0-1)
            top_k: количество результатов для оценки
            displacement: смещение для начала оценки
        """
        df = data
        total = 0
        correct = 0
        both = 0
        present = 0

        for i in tqdm(range(displacement, df.shape[0]), desc="Оценка гибридного поиска"):
            # Пропускаем пустые вопросы
            if pd.isna(df.iloc[i][column_name]) or df.iloc[i][column_name] == "-":
                continue

            query = df.iloc[i][column_name]

            # Гибридный поиск
            fused_scores = hybrid_scores(
                query,
                self.dense_retriever,
                self.sparse_retriever,
                self.data_chunks,
                alpha=alpha,
                top_k_dense=100,
                top_k_sparse=100,
                top_k_fused=top_k
            )

            if not fused_scores:
                continue

            # Получаем id_ch результатов
            retrieved_doc_ids = list(fused_scores.keys())

            # Преобразуем id_ch в chapter_index
            curr_chapter_indexes = []
            for doc_id in retrieved_doc_ids:
                chapter_idx = self.id_to_chapter_map.get(doc_id)
                if chapter_idx is not None:
                    # Преобразуем в int для сравнения
                    try:
                        curr_chapter_indexes.append(int(chapter_idx))
                    except (ValueError, TypeError):
                        pass

            # Ожидаемый индекс главы - это просто индекс строки в data
            expected_chapter_index = i

            # Оценка
            total += 1

            if curr_chapter_indexes:
                # Top-1
                if expected_chapter_index == curr_chapter_indexes[0]:
                    correct += 1

                # Top-2
                if len(curr_chapter_indexes) >= 2:
                    if expected_chapter_index in curr_chapter_indexes[:2]:
                        both += 1
                elif expected_chapter_index == curr_chapter_indexes[0]:
                    both += 1

                # Top-K
                if expected_chapter_index in curr_chapter_indexes[:top_k]:
                    present += 1

        # Результаты
        self.result["total"] = total
        self.result["correct"] = correct
        self.result["both"] = both
        self.result["present"] = present

        if total > 0:
            self.result["top1"] = (correct / total) * 100
            self.result["top2"] = (both / total) * 100
            self.result["top5"] = (present / total) * 100
        else:
            self.result["top1"] = 0.0
            self.result["top2"] = 0.0
            self.result["top5"] = 0.0

        return self.result

    def show_result(self):
        """Вывод результатов оценки"""
        print(f"\n{'='*60}")
        print(f"РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА")
        print(f"{'='*60}")
        print(f'Вопросов всего: {self.result["total"]}')
        print(f'Правильных индексов (первый верный): {self.result["correct"]} | '
              f'Доля: {self.result["top1"]:.2f}%')
        print(f'Первый или второй верные: {self.result["both"]} | '
              f'Доля: {self.result["top2"]:.2f}%')
        print(f'Индекс есть в списке из 5: {self.result["present"]} | '
              f'Доля: {self.result["top5"]:.2f}%')
        print(f"{'='*60}\n")

In [ ]:
hybrid_evaluation = HybridSearchEvaluation(
    dense_retriever=vector_store_serv_from_client,
    sparse_retriever=bm25,
    data_chunks=data_chunks
)

In [ ]:
print("Тестирование различных значений alpha:\n")

alphas = [0.0, 0.3, 0.5, 0.7, 1.0]
results_comparison = []

for alpha in alphas:
    print(f"\n{'='*60}")
    print(f"Alpha = {alpha} (Dense: {alpha*100:.0f}%, Sparse: {(1-alpha)*100:.0f}%)")
    print(f"{'='*60}")

    res = hybrid_evaluation.evaluate(data, "question", alpha=alpha, top_k=5)
    hybrid_evaluation.show_result()

    results_comparison.append({
        'alpha': alpha,
        'top1': res['top1'],
        'top2': res['top2'],
        'top5': res['top5']
    })

# Сравнительная таблица
print("\n" + "="*80)
print("СРАВНИТЕЛЬНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
print("="*80)
print(f"{'Alpha':<10} {'Dense%':<10} {'Sparse%':<10} {'Top-1':<15} {'Top-2':<15} {'Top-5':<15}")
print("-"*80)

for r in results_comparison:
    print(f"{r['alpha']:<10.1f} {r['alpha']*100:<10.0f} {(1-r['alpha'])*100:<10.0f} "
          f"{r['top1']:<15.2f} {r['top2']:<15.2f} {r['top5']:<15.2f}")

print("="*80)

# Находим лучшее значение alpha
best_alpha = max(results_comparison, key=lambda x: x['top5'])
print(f"\nЛучший результат при alpha = {best_alpha['alpha']:.1f}")
print(f"Top-1: {best_alpha['top1']:.2f}%, Top-2: {best_alpha['top2']:.2f}%, "
      f"Top-5: {best_alpha['top5']:.2f}%")

Тестирование различных значений alpha:


Alpha = 0.0 (Dense: 0%, Sparse: 100%)


Оценка гибридного поиска: 100%|██████████| 40/40 [01:05<00:00,  1.63s/it]



РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА
Вопросов всего: 40
Правильных индексов (первый верный): 24 | Доля: 60.00%
Первый или второй верные: 29 | Доля: 72.50%
Индекс есть в списке из 5: 35 | Доля: 87.50%


Alpha = 0.3 (Dense: 30%, Sparse: 70%)


Оценка гибридного поиска: 100%|██████████| 40/40 [01:00<00:00,  1.52s/it]



РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА
Вопросов всего: 40
Правильных индексов (первый верный): 24 | Доля: 60.00%
Первый или второй верные: 29 | Доля: 72.50%
Индекс есть в списке из 5: 35 | Доля: 87.50%


Alpha = 0.5 (Dense: 50%, Sparse: 50%)


...


РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА
Вопросов всего: 40
Правильных индексов (первый верный): 24 | Доля: 60.00%
Первый или второй верные: 29 | Доля: 72.50%
Индекс есть в списке из 5: 32 | Доля: 80.00%


Alpha = 0.7 (Dense: 70%, Sparse: 30%)


Оценка гибридного поиска: 100%|██████████| 40/40 [01:01<00:00,  1.53s/it]



РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА
Вопросов всего: 40
Правильных индексов (первый верный): 12 | Доля: 30.00%
Первый или второй верные: 12 | Доля: 30.00%
Индекс есть в списке из 5: 12 | Доля: 30.00%


Alpha = 1.0 (Dense: 100%, Sparse: 0%)


Оценка гибридного поиска: 100%|██████████| 40/40 [01:00<00:00,  1.51s/it]


РЕЗУЛЬТАТЫ ОЦЕНКИ ГИБРИДНОГО ПОИСКА
Вопросов всего: 40
Правильных индексов (первый верный): 0 | Доля: 0.00%
Первый или второй верные: 0 | Доля: 0.00%
Индекс есть в списке из 5: 0 | Доля: 0.00%


СРАВНИТЕЛЬНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ
Alpha      Dense%     Sparse%    Top-1           Top-2           Top-5          
--------------------------------------------------------------------------------
0.0        0          100        60.00           72.50           87.50          
0.3        30         70         60.00           72.50           87.50          
0.5        50         50         60.00           72.50           80.00          
0.7        70         30         30.00           30.00           30.00          
1.0        100        0          0.00            0.00            0.00           

Лучший результат при alpha = 0.0
Top-1: 60.00%, Top-2: 72.50%, Top-5: 87.50%
